In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
import numpy as np

conf = SparkConf().setMaster('local').setAppName('Mi programa')
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

# Base de datos reemezclada, reducida y limpiada con esta código
dfspark = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').load('./base_datos_2008.csv',nrows=1e6)
dfspark = dfspark.sample(fraction = 0.001, withReplacement = False)
dfspark = dfspark.withColumn('ArrDelay',dfspark['ArrDelay'].cast('integer'))

# Checar que no hay valores faltantes
df2 = dfspark.na.drop(subset = ['ArrDelay','DepDelay','Distance'])
# Checar que no hay valores nulos
df2 = df2.filter('ArrDelay is not NULL')
# Checar que no hay valores duplicados
df2 = df2.dropDuplicates()

In [2]:
# Primer ejemplo de acción
# Crear tablas de resumen de los principales estadísticos cuantitativos para una columna de nuestros DataFrames
# Permite hacernos una idea general de cómo está distribuida la varaible, especialmente en relación a sus colas.
# Se observa que la variable tiene mucha más carga hacia la derecha como se puede ver explorando el máximo y el mínimo
df2.select('ArrDelay').describe().show()

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|              7019|
|   mean|  8.84584698675025|
| stddev|38.130180798456095|
|    min|               -61|
|    max|               469|
+-------+------------------+



In [3]:
# Segundo ejemplo
# Obtener una lista de ocurrencias para una variable categórica.
# Se aplica el método countByValue sobre un rdd, no sobre la columna directamente
# Aparentemente se ordenan según el orden de aparición
# Lo que justifica que al inicio estén las categorías más pobladas y que aparezcan mucho
# Al final son variables muy escasas lo que lo hace menos probable que aparezcan
df2.select('Origin').rdd.countByValue()

defaultdict(int,
            {Row(Origin='ELP'): 33,
             Row(Origin='MKE'): 46,
             Row(Origin='ORD'): 351,
             Row(Origin='DEN'): 276,
             Row(Origin='MEM'): 85,
             Row(Origin='DFW'): 276,
             Row(Origin='AUS'): 40,
             Row(Origin='SRQ'): 5,
             Row(Origin='SAV'): 17,
             Row(Origin='BOS'): 100,
             Row(Origin='SNA'): 40,
             Row(Origin='DTW'): 146,
             Row(Origin='LGA'): 130,
             Row(Origin='DAL'): 46,
             Row(Origin='JAX'): 29,
             Row(Origin='PDX'): 68,
             Row(Origin='HPN'): 12,
             Row(Origin='SLC'): 128,
             Row(Origin='IAH'): 184,
             Row(Origin='EWR'): 148,
             Row(Origin='LAX'): 235,
             Row(Origin='OMA'): 30,
             Row(Origin='ALB'): 17,
             Row(Origin='SAN'): 112,
             Row(Origin='DCA'): 81,
             Row(Origin='MCI'): 67,
             Row(Origin='MSY'): 37,
 

In [5]:
# Tercer lugar
# Obtención de estadísticos de forma aislada (sin la función describe)
df2.select('ArrDelay').rdd.max()
# Si se desea obtener sólo el número en concreto se selecciona la siguiente instrucción
df2.select('ArrDelay').rdd.max()[0]

469

In [6]:
# Cuarto lugar
# Collect es una de las funciones más genéricas que se puede usar para tener una lista completa de la columna
# Si se quisiera obtener los valores únicos de esta columna se tendría que usar ademas el distinct
df2.select('Origin').rdd.collect()

[Row(Origin='ELP'),
 Row(Origin='MKE'),
 Row(Origin='ORD'),
 Row(Origin='DEN'),
 Row(Origin='MEM'),
 Row(Origin='DFW'),
 Row(Origin='AUS'),
 Row(Origin='ORD'),
 Row(Origin='SRQ'),
 Row(Origin='SAV'),
 Row(Origin='BOS'),
 Row(Origin='SNA'),
 Row(Origin='DTW'),
 Row(Origin='LGA'),
 Row(Origin='DAL'),
 Row(Origin='JAX'),
 Row(Origin='PDX'),
 Row(Origin='HPN'),
 Row(Origin='SLC'),
 Row(Origin='ORD'),
 Row(Origin='IAH'),
 Row(Origin='SLC'),
 Row(Origin='EWR'),
 Row(Origin='SLC'),
 Row(Origin='LAX'),
 Row(Origin='BOS'),
 Row(Origin='IAH'),
 Row(Origin='OMA'),
 Row(Origin='ALB'),
 Row(Origin='SAN'),
 Row(Origin='DCA'),
 Row(Origin='MCI'),
 Row(Origin='MSY'),
 Row(Origin='PIT'),
 Row(Origin='PFN'),
 Row(Origin='MDW'),
 Row(Origin='CHS'),
 Row(Origin='ITO'),
 Row(Origin='SJC'),
 Row(Origin='MOB'),
 Row(Origin='BDL'),
 Row(Origin='DEN'),
 Row(Origin='DEN'),
 Row(Origin='PHX'),
 Row(Origin='MCI'),
 Row(Origin='ORD'),
 Row(Origin='CLT'),
 Row(Origin='MSP'),
 Row(Origin='ATL'),
 Row(Origin='TPA'),


In [7]:
# Tablas de contingencia entre dos variables
# Muestra cuáles son los aeropuertos con más salidas agrupado con los días de la semana
df2.crosstab('DayOfWeek','Origin').take(2)
# Se interpreta cuántas veces se ha salido de cierto aeropuerto respecto a los días de la semana

[Row(DayOfWeek_Origin='5', ABE=1, ABI=0, ABQ=4, ACT=0, ACV=0, AEX=0, AGS=0, ALB=1, AMA=1, ANC=4, ASE=0, ATL=69, ATW=0, AUS=5, AVL=1, AVP=1, AZO=0, BDL=3, BET=1, BFL=0, BHM=0, BIL=2, BJI=0, BMI=0, BNA=6, BOI=3, BOS=10, BQK=1, BQN=0, BRO=0, BRW=1, BTM=1, BTR=0, BTV=0, BUF=2, BUR=0, BWI=15, BZN=0, CAE=3, CAK=2, CDC=0, CDV=0, CHA=0, CHO=1, CHS=2, CIC=1, CID=1, CLD=0, CLE=8, CLT=18, CMH=5, COD=0, COS=5, CRP=0, CRW=0, CVG=11, CWA=0, DAB=0, DAL=6, DAY=1, DBQ=0, DCA=8, DEN=47, DFW=31, DHN=1, DLH=0, DRO=0, DSM=3, DTW=24, EGE=0, EKO=0, ELM=1, ELP=2, EUG=0, EVV=1, EWN=0, EWR=21, EYW=0, FAI=0, FAR=1, FAT=3, FAY=0, FCA=1, FLG=0, FLL=16, FNT=1, FSD=1, FWA=1, GCC=0, GEG=1, GFK=1, GGG=0, GJT=0, GNV=0, GPT=0, GRB=0, GRK=1, GRR=1, GSO=1, GSP=1, GTF=0, HDN=2, HHH=0, HLN=0, HNL=11, HOU=5, HPN=4, HRL=1, HSV=0, HTS=0, IAD=9, IAH=30, ICT=4, IDA=0, ILM=1, IND=6, IPL=0, ISP=4, ITO=0, IYK=0, JAC=0, JAN=0, JAX=5, JFK=18, JNU=0, KOA=1, KTN=0, LAN=1, LAS=26, LAX=27, LBB=2, LCH=0, LEX=3, LFT=2, LGA=21, LGB=4, LIH=2